<a href="https://colab.research.google.com/github/plaban1981/Hugging_Face_transformers_topics/blob/main/mosaicml_mpt_7b_instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU -e git+https://github.com/samhavens/just-triton-flash.git#egg=flash_attn 

  Preparing metadata (setup.py) ... done


In [2]:
!pip install -qU  einops transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.6 MB/s eta 0:00:00


In [3]:
import os
from threading import Event, Thread
from uuid import uuid4

#import gradio as gr
import requests
import torch
from transformers import StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
#
from typing import Any, Dict, Tuple
import warnings

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:

INSTRUCTION_KEY = "### Instruction:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"
INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
PROMPT_FOR_GENERATION_FORMAT = """{intro}
{instruction_key}
{instruction}
{response_key}
""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
)


class InstructionTextGenerationPipeline:
    def __init__(
        self,
        model_name,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        use_auth_token=None,
    ) -> None:
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch_dtype,
            trust_remote_code=trust_remote_code,
            use_auth_token=use_auth_token,
        )

        tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            trust_remote_code=trust_remote_code,
            use_auth_token=use_auth_token,
        )
        if tokenizer.pad_token_id is None:
            warnings.warn(
                "pad_token_id is not set for the tokenizer. Using eos_token_id as pad_token_id."
            )
            tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "left"
        self.tokenizer = tokenizer

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.eval()
        self.model.to(device=device, dtype=torch_dtype)

        self.generate_kwargs = {
            "temperature": 0.5,
            "top_p": 0.92,
            "top_k": 0,
            "max_new_tokens": 512,
            "use_cache": True,
            "do_sample": True,
            "eos_token_id": self.tokenizer.eos_token_id,
            "pad_token_id": self.tokenizer.pad_token_id,
            "repetition_penalty": 1.1,  # 1.0 means no penalty, > 1.0 means penalty, 1.2 from CTRL paper
        }

    def format_instruction(self, instruction):
        return PROMPT_FOR_GENERATION_FORMAT.format(instruction=instruction)

    def __call__(
        self, instruction: str, **generate_kwargs: Dict[str, Any]
    ) -> Tuple[str, str, float]:
        s = PROMPT_FOR_GENERATION_FORMAT.format(instruction=instruction)
        input_ids = self.tokenizer(s, return_tensors="pt").input_ids
        input_ids = input_ids.to(self.model.device)
        gkw = {**self.generate_kwargs, **generate_kwargs}
        with torch.no_grad():
            output_ids = self.model.generate(input_ids, **gkw)
        # Slice the output_ids tensor to get only new tokens
        new_tokens = output_ids[0, len(input_ids[0]) :]
        output_text = self.tokenizer.decode(new_tokens, skip_special_tokens=True)
        return output_text

In [5]:
from getpass import getpass
import os
#
os.environ["HF_TOKEN"] = getpass()
# Configuration
HF_TOKEN = os.getenv("HF_TOKEN", None)

··········


In [6]:
# Define a custom stopping criteria
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

In [8]:
# Initialize the model and tokenizer
generate = InstructionTextGenerationPipeline(
    "mosaicml/mpt-7b-instruct",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    use_auth_token=HF_TOKEN,
)
stop_token_ids = generate.tokenizer.convert_tokens_to_ids(["<|endoftext|>"])


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- configuration_mpt.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- norm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- attention.py
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- blocks.py
- norm.py
- attention.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- meta_init_context.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- hf_prefixlm_converter.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- param_init_fns.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- adapt_tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- modeling_mpt.py
- blocks.py
- meta_init_context.py
- hf_prefixlm_converter.py
- param_init_fns.py
- adapt_tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


/root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-instruct/a858cfabdc6bf69c03ce63236a5e877517bb957c/attention.py:157: UserWarning: Using `attn_impl: torch`. If your model does not use `alibi` or `prefix_lm` we recommend using `attn_impl: flash` otherwise we recommend using `attn_impl: triton`.
  warnings.warn('Using `attn_impl: torch`. If your model does not use `alibi` or ' + '`prefix_lm` we recommend using `attn_impl: flash` otherwise ' + 'we recommend using `attn_impl: triton`.')


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<ipython-input-4-3d1d233f3f05>:38: UserWarning: pad_token_id is not set for the tokenizer. Using eos_token_id as pad_token_id.
  warnings.warn(


In [10]:
import datetime
def log_conversation(session_id, instruction, response, generate_kwargs):
    logging_url = os.getenv("LOGGING_URL", None)
    if logging_url is None:
        return

    timestamp = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")

    data = {
        "session_id": session_id,
        "timestamp": timestamp,
        "instruction": instruction,
        "response": response,
        "generate_kwargs": generate_kwargs,
    }

    try:
        requests.post(logging_url, json=data)
    except requests.exceptions.RequestException as e:
        print(f"Error logging conversation: {e}")

In [22]:
instruction="Explain how photosynthesis works to a 6 year old in a few sentences."

In [25]:
tokenized_input = generate.tokenizer(generate.format_instruction(instruction), return_tensors="pt")

In [26]:
tokenized_input.keys()

dict_keys(['input_ids', 'attention_mask'])

In [27]:
stop = StopOnTokens()

In [30]:
outputs = generate.model.generate(input_ids=tokenized_input['input_ids'].to('cuda:0'), 
                            max_new_tokens=150, 
                            temperature =0.1,
                            top_k=5, 
                            top_p=0.95,
                            stopping_criteria =StoppingCriteriaList([stop]))

In [32]:
answer = generate.tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [33]:
print(answer[0].rstrip())

Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Explain how photosynthesis works to a 6 year old in a few sentences.
### Response:
Photosynthesis is a process that plants use to convert light energy into chemical energy.  The process starts with the absorption of light energy by chlorophyll in the leaves.  This energy is then used to convert carbon dioxide and water into oxygen and sugar.  The oxygen is released into the atmosphere and the sugar is used by the plant for growth and respiration.


In [34]:
instruct = """You are a Quiz Generator assitant.Create a 5 question multiple choice quiz on the context specified below and provide solutions. 
    CONTEXT :
    
    """

In [35]:
prompt = f"""{instruct}
Google LLC is an American multinational technology company focusing on online advertising, search engine technology, cloud computing, computer software, quantum computing, e-commerce, artificial intelligence, and consumer electronics. It has been referred to as "the most powerful company in the world" and one of the world's most valuable brands due to its market dominance, data collection, and technological advantages in the area of artificial intelligence.Its parent company Alphabet is considered one of the Big Five American information technology companies, alongside Amazon, Apple, Meta, and Microsoft.Google was founded in 1998 by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University in California. Together they own about 14 percent of its shares and control 56 percent of the stockholder voting power through supervoting stock. They incorporated Google as a privately held company on September 4, 1998. An initial public offering (IPO) took place on August 19, 2004, and Google moved to its headquarters in Mountain View, California, nicknamed the Googleplex. In August 2015, Google announced plans to reorganize its various interests as a conglomerate called Alphabet Inc. Google is Alphabet's leading subsidiary and will continue to be the umbrella company for Alphabet's Internet interests. Sundar Pichai was appointed CEO of Google, replacing Larry Page who became the CEO of Alphabet."""

In [36]:
prompt

'You are a Quiz Generator assitant.Create a 5 question multiple choice quiz on the context specified below and provide solutions. \n    CONTEXT :\n    \n    \nGoogle LLC is an American multinational technology company focusing on online advertising, search engine technology, cloud computing, computer software, quantum computing, e-commerce, artificial intelligence, and consumer electronics. It has been referred to as "the most powerful company in the world" and one of the world\'s most valuable brands due to its market dominance, data collection, and technological advantages in the area of artificial intelligence.Its parent company Alphabet is considered one of the Big Five American information technology companies, alongside Amazon, Apple, Meta, and Microsoft.Google was founded in 1998 by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University in California. Together they own about 14 percent of its shares and control 56 percent of the stockholder voting power

In [48]:
def generate_results(prompt):
  print(f"Prompt Generated \n:{generate.format_instruction(prompt)}")
  tokenized_input = generate.tokenizer(generate.format_instruction(prompt), return_tensors="pt")
  #
  stop = StopOnTokens()
  #
  outputs = generate.model.generate(input_ids=tokenized_input['input_ids'].to('cuda:0'), 
                            max_new_tokens=200, 
                            temperature =0.1,
                            top_k=5, 
                            top_p=0.95,
                            stopping_criteria =StoppingCriteriaList([stop]))
  #
  answer = generate.tokenizer.batch_decode(outputs, skip_special_tokens=True)
  print(answer[0].rstrip())
  return answer[0].rstrip()

In [41]:
results = generate_results(prompt)

Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
You are a Quiz Generator assitant.Create a 5 question multiple choice quiz on the context specified below and provide solutions. 
    CONTEXT :
    
    
Google LLC is an American multinational technology company focusing on online advertising, search engine technology, cloud computing, computer software, quantum computing, e-commerce, artificial intelligence, and consumer electronics. It has been referred to as "the most powerful company in the world" and one of the world's most valuable brands due to its market dominance, data collection, and technological advantages in the area of artificial intelligence.Its parent company Alphabet is considered one of the Big Five American information technology companies, alongside Amazon, Apple, Meta, and Microsoft.Google was founded in 1998 by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University i

In [43]:
print(results.split("### Response:")[-1]

'\n1. Google is an American multinational technology company focusing on online advertising, search engine technology, cloud computing, computer software, quantum computing, e-commerce, artificial intelligence, and consumer electronics.\n2. It has been referred to as "the most powerful company in the world" and one of the world\'s most valuable brands due to its market dominance, data collection, and technological advantages in the area of artificial intelligence.\n3. Its parent company Alphabet is considered one of the Big Five American information technology companies, alongside Amazon, Apple, Meta, and Microsoft.\n4. Google was founded in 1998 by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University in California. Together they own about 14 percent of its shares and control 56 percent of the stockholder voting power through supervoting stock.\n5. They incorporated Google as a privately held company on September 4, 1998. An initial public offering (IPO) too

#### Example 3:

In [46]:
 prompt = """ You are a teaching assitant.Please provide extremely clear and detailed explanations about the topic specified below <like a 5 year old.
Also please provide sample examples to help the student understand the topic better. 
TOPIC :
What is Retrieval Enhanced Generative Question Answering  ?
"""

In [47]:
prompt  

' You are a teaching assitant.Please provide extremely clear and detailed explanations about the topic specified below <like a 5 year old.\nAlso please provide sample examples to help the student understand the topic better. \nTOPIC :\nWhat is Retrieval Enhanced Generative Question Answering  ?\n'

In [49]:
results = generate_results(prompt)

Prompt Generated 
:Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
 You are a teaching assitant.Please provide extremely clear and detailed explanations about the topic specified below <like a 5 year old.
Also please provide sample examples to help the student understand the topic better. 
TOPIC :
What is Retrieval Enhanced Generative Question Answering  ?

### Response:

Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
 You are a teaching assitant.Please provide extremely clear and detailed explanations about the topic specified below <like a 5 year old.
Also please provide sample examples to help the student understand the topic better. 
TOPIC :
What is Retrieval Enhanced Generative Question Answering ?

### Response:
Retrieval-enhanced generative question answering (REGA) is a machine learning technique for generating answers to questio

In [52]:
print(results.split("### Response:")[-1])


Retrieval-enhanced generative question answering (REGA) is a machine learning technique for generating answers to questions. It is a form of generative question answering (QA), which is a machine learning technique for generating answers to questions.

In REGA, a question is first encoded as a vector, and then a retrieval model is used to retrieve relevant documents from a large corpus. The retrieved documents are then encoded as vectors, and a generative model is used to generate an answer from the retrieved documents.

REGA has been shown to outperform other generative QA techniques on several datasets.


#### Example : 4

In [53]:
prompt = f""" Translate the below text from English to Spanish.
TEXT:
I had a very good day today.
"""

In [54]:
results = generate_results(prompt)

Prompt Generated 
:Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
 Translate the below text from English to Spanish.
TEXT:
I had a very good day today.

### Response:

Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
 Translate the below text from English to Spanish.
TEXT:
I had a very good day today.

### Response:
Hoy tuve un muy buen dia.


In [55]:
print(results.split("### Response:")[-1])


Hoy tuve un muy buen dia.
